In [1]:
#read file 
import pandas as pd
import requests
import json
from config import gkey

filepath = "pollution_us_2000_2016.csv"
us_pollution_data = pd.read_csv(filepath)
us_pollution_data.head()


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [2]:
#Create Year column. which we will use to group the data by year.
Date_data = []
#run a loop on the 'Date Local' Column
for i in us_pollution_data['Date Local']:
#Split each row (which is date in string format) into a list
    date = i.split("-")
#The first element of the list will be the year.
    year= date[0]
#Append the year to a list of dates
    Date_data.append(year)
#Create a year column using Date data
us_pollution_data['Year'] = Date_data

#clean data by dropping rows with null values.
us_pollution_data_clean = us_pollution_data.dropna(how='any')

In [24]:
#Create a collection of columns of interest 
Chem_Columns_data = us_pollution_data_clean[['State', 'Date Local', 'Year','NO2 Units', 'NO2 Mean', 'O3 Units', 'O3 Mean','SO2 Units', 'SO2 Mean', 'CO Units', 'CO Mean', 'SO2 AQI', 'CO AQI', 'O3 AQI', 'NO2 AQI']]
#reindex data to set indices starting from 0 
Chem_Columns_data = Chem_Columns_data.reset_index()
#drop the index column 
Chem_Columns_data = Chem_Columns_data.drop(columns=['index'])
Chem_Columns_data.head()

,State,Date Local,Year,NO2 Units,NO2 Mean,O3 Units,O3 Mean,SO2 Units,SO2 Mean,CO Units,CO Mean,SO2 AQI,CO AQI,O3 AQI,NO2 AQI
0,Arizona,2000-01-01,2000,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,Parts per million,0.878947,13.0,25.0,34,46
1,Arizona,2000-01-02,2000,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,Parts per million,1.066667,4.0,26.0,27,34
2,Arizona,2000-01-03,2000,Parts per billion,38.125000,Parts per million,0.007958,Parts per billion,5.250000,Parts per million,1.762500,16.0,28.0,14,48
3,Arizona,2000-01-04,2000,Parts per billion,40.260870,Parts per million,0.014167,Parts per billion,7.083333,Parts per million,1.829167,23.0,34.0,28,72
4,Arizona,2000-01-05,2000,Parts per billion,48.450000,Parts per million,0.006667,Parts per billion,8.708333,Parts per million,2.700000,21.0,42.0,10,58


In [25]:
#convert mean values to same units.
#In this case we convert all parts per billion to parts per million. 
#To do this we divide all parts per billion observations by 1000
Chem_Columns_data['NO2 Mean'] = Chem_Columns_data['NO2 Mean']/1000
Chem_Columns_data['SO2 Mean'] = Chem_Columns_data['SO2 Mean']/1000

In [26]:
Chem_by_ST_Year = Chem_Columns_data.groupby(['Year', 'State']).mean()
Chem_by_ST_Year

NO2 Mean   O3 Mean  SO2 Mean   CO Mean    SO2 AQI  \
Year State                                                                     
2000 Arizona               0.026492  0.023664  0.002298  0.746075   7.479545   
     California            0.017581  0.023675  0.001730  0.647508   6.045758   
     Colorado              0.014902  0.017066  0.002114  0.615031  21.509589   
     District Of Columbia  0.022726  0.017567  0.008172  1.178786  24.774725   
     Florida               0.012453  0.026261  0.001983  0.726884   7.556180   
     Illinois              0.026437  0.017105  0.006099  0.706236  24.583807   
     Indiana               0.015288  0.029782  0.007299  0.875499  24.934132   
     Kansas                0.009704  0.029726  0.000952  0.756692   5.651762   
     Kentucky              0.012661  0.028916  0.004770  0.178344  22.781592   
     Louisiana             0.017346  0.023756  0.003664  0.758775  20.010929   
     Michigan              0.022682  0.023367  0.005421  0.280681  30.666667   
     Missouri              0.016873  0.027761  0.004167  0.524458  20.733179   
     New Jersey            0.020528  0.020263  0.006294  0.586453  21.122905   
     New York              0.023152  0.021248  0.007198  0.464648  21.138004   
     North Carolina        0.015173  0.034652  0.003691  0.619122  18.284530   
     Oklahoma              0.004762  0.037350  0.000850  0.047210   8.317164   
     Pennsylvania          0.016129  0.024542  0.005174  0.303847  21.555418   
     Texas                 0.016331  0.024600  0.002767  0.437538  14.282587   
     Virginia              0.017918  0.022604  0.009071  0.647606  24.589020   
2001 Arizona               0.022741  0.021632  0.001674  0.652423   5.882159   
     California            0.017759  0.024845  0.001823  0.625402   6.333083   
     Colorado              0.025783  0.016376  0.002424  0.630548  23.987539   
     District Of Columbia  0.024474  0.020936  0.006700  1.077945  23.011461   
     Florida               0.012363  0.024173  0.001155  0.640209   6.167614   
     Illinois              0.022357  0.020868  0.005501  0.593802  27.491741   
     Indiana               0.014948  0.032153  0.003276  0.383134  18.786517   
     Kansas                0.010595  0.027978  0.001003  0.336699   5.313673   
     Kentucky              0.012548  0.030233  0.004072  0.131677  21.221154   
     Louisiana             0.017359  0.022978  0.003694  0.827924  18.547222   
     Massachusetts         0.024048  0.019194  0.005924  0.300087  16.153846   
...                             ...       ...       ...       ...        ...   
2016 Colorado              0.023624  0.020989  0.001015  0.453770   3.947955   
     Delaware              0.013771  0.025159  0.000844  0.284438   1.952381   
     District Of Columbia  0.012461  0.031057  0.000989  0.256361   2.206897   
     Florida               0.005493  0.029213  0.000399  0.574539   0.813187   
     Georgia               0.011935  0.019770  0.000298  0.315240   0.555556   
     Illinois              0.010374  0.025014  0.000957  0.310079   3.145251   
     Indiana               0.013857  0.026542  0.000404  0.245990   1.381443   
     Iowa                  0.005995  0.030609  0.000197  0.211936   0.296703   
     Kansas                0.011582  0.022840  0.001098  0.251962   2.750000   
     Louisiana             0.012911  0.026846  0.001123  0.234943   2.964912   
     Maine                 0.006557  0.028197  0.000345  0.300699   1.587413   
     Maryland              0.009201  0.027409  0.000746  0.250659   1.988806   
     Massachusetts         0.015829  0.023563  0.000692  0.270368   1.021978   
     Missouri              0.012486  0.023046  0.001131  0.324271   2.693878   
     Nevada                0.017421  0.022641  0.000392  0.322915   1.512821   
     New Hampshire         0.004294  0.031651  0.000835  0.212214   1.237288   
     New York              0.017847  0.023972  0.001068  0.295042   2.686192   
     North Carolina        

In [27]:
#Create a column for the average of the mean measurements
#note still need to take conversions into account
Mean_us_pol = []
for index, row in Chem_by_ST_Year.iterrows():
    avg_pol = (row[0] + row[1] + row[2] + row[3])/4
    Mean_us_pol.append(avg_pol)
    
Chem_by_ST_Year['State Average'] = Mean_us_pol

Chem_by_ST_Year.head()

NO2 Mean   O3 Mean  SO2 Mean   CO Mean    SO2 AQI  \
Year State                                                                     
2000 Arizona               0.026492  0.023664  0.002298  0.746075   7.479545   
     California            0.017581  0.023675  0.001730  0.647508   6.045758   
     Colorado              0.014902  0.017066  0.002114  0.615031  21.509589   
     District Of Columbia  0.022726  0.017567  0.008172  1.178786  24.774725   
     Florida               0.012453  0.026261  0.001983  0.726884   7.556180   

                              CO AQI     O3 AQI    NO2 AQI  State Average  
Year State                                                                 
2000 Arizona               15.169318  39.042045  47.127273       0.199632  
     California            11.268912  32.716142  29.993447       0.172624  
     Colorado              11.558904  27.553425  32.753425       0.162278  
     District Of Columbia  18.563187  27.956044  37.296703       0.306813  
     Florida               10.342697  39.219101  25.573034       0.191895

In [28]:
Chem_by_Year = Chem_Columns_data.groupby(['Year']).mean()
Chem_by_Year.head()

,NO2 Mean,O3 Mean,SO2 Mean,CO Mean,SO2 AQI,CO AQI,O3 AQI,NO2 AQI
Year,,,,,,,,
2000,0.017433,0.024376,0.003649,0.557789,14.260908,9.540096,36.022688,30.969852
2001,0.017600,0.025388,0.003133,0.509119,12.836909,8.792393,37.641191,31.310673
2002,0.016387,0.026818,0.002744,0.500990,11.030905,8.486407,39.559160,29.363726
2003,0.015865,0.025521,0.002865,0.492720,11.328351,8.246634,36.631114,28.334598
2004,0.014968,0.024914,0.002855,0.448132,11.249550,7.411070,34.741209,26.940324


In [29]:
Year_Mean_us_pol = []
for index, row in Chem_by_Year.iterrows():
    avg_pol = (row[0] + row[1] + row[2] + row[3])/4
    Year_Mean_us_pol.append(avg_pol)
    
Chem_by_Year['Year Average'] = Year_Mean_us_pol
Chem_by_Year.head()

,NO2 Mean,O3 Mean,SO2 Mean,CO Mean,SO2 AQI,CO AQI,O3 AQI,NO2 AQI,Year Average
Year,,,,,,,,,
2000,0.017433,0.024376,0.003649,0.557789,14.260908,9.540096,36.022688,30.969852,0.150812
2001,0.017600,0.025388,0.003133,0.509119,12.836909,8.792393,37.641191,31.310673,0.138810
2002,0.016387,0.026818,0.002744,0.500990,11.030905,8.486407,39.559160,29.363726,0.136735
2003,0.015865,0.025521,0.002865,0.492720,11.328351,8.246634,36.631114,28.334598,0.134243
2004,0.014968,0.024914,0.002855,0.448132,11.249550,7.411070,34.741209,26.940324,0.122717


In [30]:
#save DataFrame as Csv for easy access to Dataframe data 
Chem_by_ST_Year.to_csv("../DataFrames/Chem_mean_by_State.csv", header=True)

In [31]:
#Are we sure we have the correct interpretation of a summation of the pullutant means 

In [32]:
#save second DataFrame as Csv for easy access to Dataframe data 
Chem_by_Year.to_csv("../DataFrames/Chem_mean_by_Year.csv", header=True)

In [33]:
#Create a new Groupby object which will be help for acquiring State pollutant means by year
Group_State_then_Year = Chem_Columns_data.groupby(['State', 'Year']).mean()
Year_Mean_us_pol2 = []
#Create an average column like we did before
for index, row in Group_State_then_Year.iterrows():
    avg_pol = (row[0] + row[1] + row[2] + row[3])/4
    Year_Mean_us_pol2.append(avg_pol)
    
Group_State_then_Year['Year Average'] = Year_Mean_us_pol2
Group_State_then_Year.head()

NO2 Mean   O3 Mean  SO2 Mean   CO Mean    SO2 AQI    CO AQI  \
State   Year                                                                
Alabama 2013  0.012066  0.012099  0.000772  0.211873   6.580645  4.129032   
        2014  0.009412  0.024633  0.001198  0.206144   7.956954  3.715232   
        2015  0.009079  0.024000  0.001028  0.221390   6.950000  3.933333   
        2016  0.009525  0.028348  0.000859  0.210097   5.275168  3.906040   
Alaska  2014  0.011659  0.010307  0.005956  0.455232  14.000000  6.983240   

                 O3 AQI    NO2 AQI  Year Average  
State   Year                                      
Alabama 2013  18.903226  21.387097      0.059202  
        2014  36.947020  21.486755      0.060347  
        2015  37.343333  20.026667      0.063874  
        2016  39.295302  23.093960      0.062207  
Alaska  2014  15.206704  21.167598      0.120788

In [34]:
Group_State_then_Year.to_csv("../DataFrames/Group_State_then_Year.csv", header=True)

In [35]:
AQI_data = us_pollution_data_clean[['State Code', 'County Code', 'State', 'County', 'City','Year','NO2 AQI', 'O3 AQI', 'SO2 AQI', 'CO AQI']]
AQI_data.head()

,State Code,County Code,State,County,City,Year,NO2 AQI,O3 AQI,SO2 AQI,CO AQI
1,4,13,Arizona,Maricopa,Phoenix,2000,46,34,13.0,25.0
5,4,13,Arizona,Maricopa,Phoenix,2000,34,27,4.0,26.0
9,4,13,Arizona,Maricopa,Phoenix,2000,48,14,16.0,28.0
13,4,13,Arizona,Maricopa,Phoenix,2000,72,28,23.0,34.0
17,4,13,Arizona,Maricopa,Phoenix,2000,58,10,21.0,42.0


In [36]:
#plot the average state values for a year versus number of cases in states for that year. or 
#plot the average us_val for a year versus number of case for that year 

#might have to merge on state or produce an overall average mean (of year) for the US based on an average of all states (for a year)

In [37]:
target_city = "Phoenix, Arizona"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)

geo_data = requests.get(target_url).json()


lat = geo_data["results"][0]["geometry"]["location"]["lat"]
lng = geo_data["results"][0]["geometry"]["location"]["lng"]

In [41]:
AQI_data.to_csv("../DataFrames/AQI_data.csv", header=True)